## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

In [2]:
%load_ext line_profiler

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [3]:
import numpy as np
import numba

In [4]:
A = np.random.randint(0,1000, size=(1000000,))
A

array([426, 308, 539, ..., 275, 295, 729])

In [8]:
def f1(A):
    acc, cnt = 0,0
    for ai in A:
        bi = ai +100
        acc += bi
        cnt += 1
    return acc/ cnt
%timeit f1(A)

625 ms ± 14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
def f2(A):
    acc = 0
    for ai in A:
        bi = ai +100
        acc += bi
    return acc / len(A)
%timeit f2(A)

578 ms ± 23.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
def f3(A):
    return A.mean()+100
%timeit f2(A)

572 ms ± 14.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
@numba.njit
def f5(A):
    acc, cnt = 0,0
    for ai in A:
        bi = ai +100
        acc += bi
        cnt += 1
    return acc/cnt
%timeit f5(A)

1.08 ms ± 8.99 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [12]:
import numpy as np
import pandas as pd

df = pd.DataFrame(np.random.randint(0, 1000, size=(2_000_000, 4)),
                  columns=['col1', 'col2', 'col3', 'col4'])
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
df['key'] = np.random.choice(letters, 2_000_000, replace=True)

def g(df):
    letters = ['a', 'b', 'c', 'd', 'e']
    dfs = []
    for letter in letters:
        q = df[df['key']==letter]
        dfs.append(q)
    return pd.concat(dfs, axis=0)


def g_optimize(df):
    return df[df["key"].str.contains("a|b|c|d|e")]

def g_optimize_v2(df):
    return df[df["key"].isin(('a', 'b', 'c', 'd', 'e', 'f', 'g'))]

In [40]:
import pandas as pd
import random
import string

N = 2000000
data = {'col1': [random.randint(0, 100) for i in range(N)],
        'col2': [random.uniform(0, 1) for i in range(N)],
        'col3': [random.choice([True, False]) for i in range(N)],
        'col4': [random.choice(['A', 'B', 'C', 'D']) for i in range(N)]}

df = pd.DataFrame(data)

keys = ''.join(random.choice(string.ascii_uppercase) for _ in range(N))
df['key'] = list(keys)

subset = df[df['key'].str[:5].isin(list(string.ascii_uppercase)[:5])]

print(subset)

         col1      col2   col3 col4 key
3          43  0.082364  False    B   C
7          77  0.248854  False    D   B
12         86  0.419805  False    D   C
21         63  0.015310   True    A   B
51         68  0.159927   True    A   B
...       ...       ...    ...  ...  ..
1999975    10  0.317377  False    D   E
1999983    64  0.736150   True    D   B
1999985    60  0.940648  False    C   C
1999988     2  0.105155  False    D   B
1999993    12  0.172362   True    C   B

[383464 rows x 5 columns]


In [13]:
%timeit g(df)

1.06 s ± 26.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%timeit g_optimize(df)

1.68 s ± 22.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%timeit g_optimize_v2(df)


## Лабораторная работа 3

In [34]:
!pip install line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [16]:
import pandas as pd
import time
import numpy as np 
from numba import jit,njit


In [17]:
recipes = pd.read_csv("recipes_sample.csv", sep=",", parse_dates=['submitted'])
recipes = recipes.set_index('id')

reviews = pd.read_csv("reviews_sample.csv", sep=",",parse_dates=['date'])
reviews.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
reviews = reviews.set_index('id')

In [18]:
reviews['date'] = pd.to_datetime(reviews['date'])
reviews['rating'] = reviews['rating'].astype(float)

In [21]:
def f1_test():
    counter = 0
    values = 0
    for index, row in reviews.iterrows():
        if row["date"].year == 2010:
            values += row["rating"]
            counter += 1

    return values/counter

result1 = f1_test()
result1

4.4544402182900615

In [22]:
%timeit f1_test()

8.45 s ± 118 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
def f2_test():
    counter = 0
    values = 0
    selected_year_df = reviews[reviews['date'].dt.year == 2010]
    for index, row in selected_year_df.iterrows():
        values += row["rating"]
        counter += 1

    return values/counter
    
result2 = f2_test()
result2

4.4544402182900615

In [24]:
%timeit f2_test()

828 ms ± 16.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
def f3_test():
    selected_year_df = reviews['date'].dt.year == 2010
    return reviews.loc[selected_year_df, 'rating'].mean()

result3 = f3_test()
result3

4.4544402182900615

In [26]:
%timeit f3_test()

14.4 ms ± 91.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%lprun -f f1_test f1_test()

In [ ]:
%lprun -f f2_test f2_test()

In [ ]:
%lprun -f f3_test f3_test()

In [28]:
import numpy as np
def f4_test():
    
    data = np.matrix((reviews["rating"], reviews["date"].dt.year), dtype=int)
    #Маска
    mask = data[1,] == 2010
    #Применяем элементы для маски
    faster_values = np.where(mask, data, 0)
    #Получаем результат
    return faster_values[0,].sum() / mask.sum()

In [29]:
f4_test()

4.4544402182900615

In [30]:
%timeit f4_test()

14.3 ms ± 73.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [31]:
def get_word_reviews_count(df):
    
    word_reviews = {}
    
    #Удаляем нулевые и проходим идерацией
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        #Делаем разделение по словами 
        words = review.split(' ')
        #По каждому слову начинаем цикл
        for word in words:
            #Если слово не в словаре, то заносим его
            if word not in word_reviews:
                word_reviews[word] = []
            
            #Добавляем рецепт по этому слову
            word_reviews[word].append(recipe_id)
    
    
    word_reviews_count = {}
    #Удаляем нулевые и проходим идерацией еще разок
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        # Проходим по каждому слову
        for word in words:
            #Заносим кол-во отзывов через len 
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [32]:
def get_word_reviews_count_optimized(df):
    
    word_reviews = {}
    
    #Удаляем нулевые и проходим идерацией
    for _, row in df.dropna(subset=['review']).iterrows():

        #По каждому слову начинаем цикл
        for word in row['review'].split(' '):
            #Если слово не в словаре, то заносим его
            if word not in word_reviews:
                word_reviews[word] = 0
            #Добавляем рецепт по этому слову
            word_reviews[word] += 1
    
    return word_reviews

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [33]:
buffer = reviews[['recipe_id', 'rating']].dropna()
mask = buffer['rating'] != 0
buffer = buffer[mask].groupby(buffer['recipe_id'])['rating']

In [34]:
def executor(series_data, function):
    """Метод для вызова функции c аргументами"""
    return function(series_data.to_numpy(), series_data.mean())

In [35]:
def MAPE_FIRST(A, F):
    results_list = [abs(i - F) / i for i in A]
    return 100/len(A) * sum(results_list)

#Конвертация dataframe в series
result1 = buffer.agg(executor, MAPE_FIRST)

In [36]:
%timeit buffer.agg(executor, MAPE_FIRST)

3.2 s ± 27.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
@numba.jit(nopython=True)
def MAPE_SECOND(A, F):
    results_list = [abs(i - F) / i for i in A]
    return 100/len(A) * sum(results_list)

result2 = buffer.agg(executor, MAPE_SECOND)

In [38]:
%timeit buffer.agg(executor, MAPE_SECOND)

2.87 s ± 37.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
